## Infer schema directly & self-correct

#### Imports 

In [1]:
from vertexai.language_models import CodeGenerationModel
from google.cloud import bigquery
import logging 
import os 

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup essentials

In [3]:
SERVICE_ACCOUNT_CREDENTIALS = './../credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [4]:
PROJECT_ID = 'arun-genai-bb'
CODE_GEN_MODEL_NAME = 'code-bison@latest'
TEMPERATURE = 0 # default value = 0
MAX_OUTPUT_TOKENS = 2048  # length of the output response | overridding the default value which is 128
# TOP_P = 0.95  # default value
# TOP_K = 40  # default value
LOCATION = 'us-central1'

In [5]:
DATASET = 'flight_reservations'
TABLES = ['customers', 'flights', 'reservations', 'transactions', 'loyality_points']

In [6]:
code_gen_model = CodeGenerationModel.from_pretrained(CODE_GEN_MODEL_NAME)
bq_client = bigquery.Client()

In [ ]:
query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ({','.join([f'"{table}"' for table in TABLES])})
"""
logger.info(query)

In [ ]:
schema_columns = bq_client.query(query=query).to_dataframe()
schema_columns

In [ ]:
schema_columns = schema_columns.to_markdown(index=False)
logger.info(schema_columns)

Utility function to convert text into SQL and automatically correct the SQL query if any execution errors occur.

In [ ]:
def generate_and_execute_sql(prompt, max_tries=5):
    """
    Generate an SQL query using the code_gen_model and execute it using bq_client.
    
    Args:
    - prompt (str): Prompt to provide to the model for generating SQL.
    - max_tries (int): Maximum number of attempts to generate and execute SQL.
    
    Returns:
    - dict: A dictionary containing the successful dataframe or error messages and prompt evolution.
    """
    
    tries = 0
    error_messages = []
    prompts = [prompt]
    
    while tries < max_tries:
        logger.info(f'ATTEMPT: {tries+1}')
        try:
            # Predict SQL using the model
            response = code_gen_model.predict(prompt, temperature=TEMPERATURE, max_output_tokens=MAX_OUTPUT_TOKENS)
            generated_sql_query = response.text
            generated_sql_query = '\n'.join(generated_sql_query.split('\n')[1:-1])
            logger.info('-' * 50)
            logger.info(generated_sql_query)
            logger.info('-' * 50)
            # Execute SQL using BigQuery client
            df = bq_client.query(generated_sql_query).to_dataframe()
            logger.info('SUCCEEDED')
            # If successful, return the dataframe
            return {
                "dataframe": df,
                "prompts": prompts,
                "errors": error_messages
            }
        except Exception as e:
            logger.error('FAILED')
            # Catch the error, store the message, and try again
            msg = str(e)
            error_messages.append(msg)
            # Evolve the prompt by appending the error message and asking the model to correct it
            prompt = f"""{prompt}
Encountered an error: {msg}. 
To address this, please generate an alternative SQL query response that avoids this specific error. 
Follow the instructions mentioned above to remediate the error. 

Modify the below SQL query to resolve the issue:
{generated_sql_query}

Provide a detailed explanation of the modifications made and the rationale behind those changes. 
Ensure the revised SQL query aligns precisely with the requirements outlined in the initial question."""
            prompts.append(prompt)
            tries += 1
        logger.info('=' * 100)
    # If all tries exhausted, return the errors
    return {
        "error": "All attempts exhausted.",
        "prompts": prompts,
        "errors": error_messages
    }

### Test Text-to-SQL scenarios

Construct the SEED prompt

In [ ]:
seed_prompt = """
Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.\n
QUESTION:
{}\n
SCHEMA:
{}\n
IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
--
Ensure your SQL query accurately defines both the start and end of the DATETIME range.
"""
logger.info(seed_prompt)

#### Scenario 1: Retrieve Active Reservations for a Specific Date Range

For this scenario, you want to find all active reservations within a specific date range.

In [ ]:
question = "Provide a list of all reservations from October 10th to October 15th, 2023"

In [ ]:
prompt = seed_prompt.format(question, schema_columns)
logger.info(prompt)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

#### Scenario 2: Identify customers who made reservations in the past N days.

In [ ]:
question = "Identify all customers who have made flight reservations within the last 7 days."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

### Scenario 3: Calculate Monthly Revenue
Calculate the total revenue generated from transactions for a given month and year.

In [ ]:
question = "Calculate the total revenue generated from transactions in October 2023, specifically from all reservations with a Confirmed status."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

### Scenario 4: Popular Flight Times
Identify the most popular departure hours or days for a given day or month or year.

In [ ]:
question = "Determine the departure months with the highest frequency for the year 2023."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

### Scenario 5: Customer Age Group
Group customers by age brackets and count the number in each bracket.

In [ ]:
question = "Group customers into five distinct age brackets and count the number of customers in each bracket."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

### Scenario 6: Age Calculation
Calculate the age of customers based on their date of birth and filter those who are above X years old.

In [ ]:
question = "Identify and rank all customers aged 18 and over who have reservations with a `Confirmed` status for the current month, ordered by their age. Ensure to display their ages in the result."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)
sql_output = response['dataframe']
sql_output

Inspect the evolution of the seed prompt and how SQL query was fixed automatically by the LLM.

In [ ]:
for i, prompt in enumerate(response['prompts']):
    logger.info(f'==================== ATTEMPT {i+1} ====================')
    logger.info(prompt)